# 1. 커널/환경 점검 & 기본 경로

In [1]:
import sys, os, platform, subprocess, textwrap

print("Python:", sys.version)
print("Platform:", platform.platform())
print("Venv:", os.environ.get("VIRTUAL_ENV"))
print("CWD:", os.getcwd())

# 프로젝트 폴더에 data 폴더 없으면 생성
DATA_DIR = os.path.join(os.getcwd(), "data")
os.makedirs(DATA_DIR, exist_ok=True)
print("DATA_DIR:", DATA_DIR)

Python: 3.10.7 (tags/v3.10.7:6cc6b13, Sep  5 2022, 14:08:36) [MSC v.1933 64 bit (AMD64)]
Platform: Windows-10-10.0.19045-SP0
Venv: C:\Users\user\Practice_Rag\venv
CWD: C:\Users\user\Practice_Rag\Python_code
DATA_DIR: C:\Users\user\Practice_Rag\Python_code\data


# 2. 핵심 라이브러리 임포트 (RAG 기본)

In [2]:
# 임포트
import re, glob, json
from typing import List, Dict
import numpy as np

# 임베딩/토치
import torch
from sentence_transformers import SentenceTransformer

# 벡터 인덱스(FAISS)
import faiss

# 파일 파싱(가벼운 경로: PDF는 pypdf, 텍스트는 open)
from pypdf import PdfReader

print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

C:\Users\user\Practice_Rag\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch: 2.5.1+cu121 | CUDA available: True
GPU: NVIDIA GeForce RTX 2060


# 3. (선택) OpenAI 클라이언트 준비

In [3]:
import os
# 안전: 터미널에서 미리 설정했다면 이 셀은 생략 가능
# 예) 터미널: setx OPENAI_API_KEY "sk-XXXX"  (새 세션부터 반영)
# 노트북 임시 설정: (실서비스/버전관리에는 넣지 마세요!)
# os.environ["OPENAI_API_KEY"] = "sk-..."

from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
print("OpenAI key loaded:", bool(os.getenv("OPENAI_API_KEY")))

OpenAI key loaded: True


# 4. 데이터 가져오기 + 문서 만들기

In [6]:
import pandas as pd

In [16]:
folder = '../Data' # .. 은 폴더 뒤로가기
folder

'../Data'

In [17]:
file_name = os.listdir(folder)[0]
file_name

'보이스피싱 챗봇 즉답형 시나리오 샘플.xlsx'

In [19]:
file_path = os.path.join(folder, file_name)
file_path

'../Data\\보이스피싱 챗봇 즉답형 시나리오 샘플.xlsx'

In [21]:
data = pd.read_excel(file_path, sheet_name=None)

In [28]:
Sheets = list(data.keys())
Sheets

['전화 1차', '문자 2차', '단순상담 1차']

In [47]:
# RAG 문서 리스트 만들기
docs = []
for sheet in Sheets :
    df = data[sheet].copy()
    df = df[['문의내용', '답변내용']]
    df.loc[:, '출처'] = sheet
    for _, row in df.iterrows():
        doc = {
            "text": str(row["문의내용"]),  # 임베딩할 본문
            "metadata": {
                "답변내용": str(row["답변내용"]),
                "출처": str(row["출처"])
            }
        }
        docs.append(doc)

In [43]:
len(docs)

36

# 5. 텍스트 임베딩 하기

## 0. 환경 확인 & 장치 선택

In [57]:
import os, sys, json, time
import numpy as np
import torch

print("Python:", sys.version.split()[0])
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
device = "cuda" if torch.cuda.is_available() else "cpu"
device

Python: 3.10.7
CUDA available: True
GPU: NVIDIA GeForce RTX 2060


'cuda'

## 1. docs → LangChain Documents 변환

In [64]:
from langchain.docstore.document import Document
documents = [
    Document(page_content=d["text"], metadata=d["metadata"])
    for d in docs
]
len(documents), documents[0]

(126,
 Document(metadata={'답변내용': '대환대출이 가능하다고 하면서 기존 대출금 상환을 유도 한 후 가로채는 피싱 유형입니다. 기존 대출처인 신한카드(☎ 1544-7000)로 상환관련 내용에 대해 진위여부 확인을 하시기 바랍니다.\n단순 통화가 아닌 개인정보 노출이 있는 경우에는 명의도용 피해 예방법으로 조치하셔서 2차 피해를 예방해야 합니다.', '출처': '전화 1차'}, page_content='OK금융 담당자와 전화로 대출상담을 했는데 이후 기존 대출이 있는 신한카드 담당자한테서 계약위반을 했으니 기존 대출금을 대면상환 해야 한다면서  해지통보서 및 지급정지통보서를 보내왔어요'))

In [49]:
# intfloat/multilingual-e5-large

## 2. E5-large 임베딩 준비 (접두어 세팅)

In [66]:
from langchain_huggingface import HuggingFaceEmbeddings

# 디바이스 선택
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

class E5Embeddings(HuggingFaceEmbeddings):
    """E5 전용: query/passages 접두어 자동 부착 + L2 정규화"""
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # encode_kwargs에서 normalize_embeddings=True 설정 (코사인과 동치)
        if "encode_kwargs" not in kwargs:
            self.encode_kwargs = {"normalize_embeddings": True}

    def embed_query(self, text: str):
        return super().embed_query(f"query: {text}")

    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

embedding_model = E5Embeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={"device": device},              # "cpu" 강제 가능
    encode_kwargs={"normalize_embeddings": True}, # 안전하게 한번 더 명시
)

## 3. 벡터스토어 생성/저장 (FAISS) /  Load

In [76]:
from langchain_community.vectorstores import FAISS
try:
    from langchain_community.vectorstores.utils import DistanceStrategy
except Exception as e:
    print(e)
    from langchain_core.vectorstores import DistanceStrategy  # 일부 버전 호환

In [68]:
# 코사인 유사도: L2 정규화 + 내적
vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embedding_model,
    distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT,
    normalize_L2=True,
)

vectorstore.save_local("./faiss_index_voicephishing_1")

C:\Users\user\Practice_Rag\venv\lib\site-packages\langchain_community\vectorstores\faiss.py:233: UserWarning: Normalizing L2 is not applicable for metric type: MAX_INNER_PRODUCT
  warnings.warn(


In [69]:
vs = FAISS.load_local("./faiss_index_voicephishing_1",
                      embeddings=embedding_model,
                      allow_dangerous_deserialization=True)

## 4. 유사문서 찾기

In [79]:
query = "해외발신으로 카드가 발급됐다는 문자를 수신했는데 본인이 신청하지 않았을 경우 사고대응센터에 전화하라고 했어요"
results = vectorstore.similarity_search_with_score(query, k=3)

for i, (doc, score) in enumerate(results, 1):
    # score ∈ [0, 1] (정규화 + 내적 = 코사인유사도)
    print(f"[{i}] cosine={score:.4f} | 출처={doc.metadata.get('출처')}")
    print("문의내용:", doc.page_content[:160].replace("\n"," "), "...")
    print("답변내용:", doc.metadata.get("답변내용"))
    print("-"*80)

[1] cosine=0.9489 | 출처=문자 2차
문의내용: 해외발신으로 카드가 발급됐다는 문자를 수신했는데 본인이 신청하지 않았을 경우 사고대응센터에 전화하라고 했어요 ...
답변내용: 정상적인 발급 문자라면 카드사 정보가 있어야 합니다. 발신번호 또는 문자내 안내처가 아닌 해당 카드사의 대표번호로 전화를 하여 발급 유무를 확인해야 하며, 카드사 정보가 없는 경우에는 전화를 유도하여 개인정보 탈취를 하려는 미끼문자일 가능성이 높습니다. 
안드로이드용 사용자는 메시지신고 > 피싱으로 신고 후 문자 삭제를 하고 발신번호도 차단을 하시기 바랍니다. iOS사용자는 발신번호 차단 후 삭제하시기 바랍니다.
--------------------------------------------------------------------------------
[2] cosine=0.9190 | 출처=문자 2차
문의내용: 해외발신으로 카드가 발급됐다는 문자를 수신했는데 본인이 신청하지 않았을 경우 전화하라고 해서 통화하여 인증번호를 말했어요 ...
답변내용: 노출된 인증번호의 사용처가 어디인지에 따라 개인정보 노출 피해 상황이 달라질 수 있기 때문에 명의도용 유무 확인방법 1번~3번까지 조치하여 파악하시기 바랍니다.
또한 추후 일어날 수 있는 2차피해 예방을 위해 명의도용 예방법 1번 ~ 4번까지 조치하시기 바랍니다.
--------------------------------------------------------------------------------
[3] cosine=0.9059 | 출처=문자 2차
문의내용: 카드가 발급되었다는 해외발신 문자를 수신했는데 문의사항이 있으면 전화하라는 번호가 있어요 ...
답변내용: 정상적인 발급 문자라면 카드사 정보가 있어야 합니다. 발신번호 또는 문자내 안내처가 아닌 해당 카드사의 대표번호로 전화를 하여 발급 유무를 확인해야 하며, 카드사 정보가 없는 경우에는 전화를 유도하여 개인정보 탈취를 하려는 미끼문자일 가능성이 높습니다. 


# 6. OpenAI API + context 필터링 코드

In [81]:
# 환경변수에 OPENAI_API_KEY 설정되어 있어야 함
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [86]:
def build_prompt(query, results, threshold=0.8):
    global context
    """코사인 유사도 threshold 이상인 문서만 context로 포함"""
    filtered = [(doc, score) for doc, score in results if score >= threshold]

    if not filtered:
        return f"사용자 질문: {query}\n\n관련 문서가 없어 답변할 수 없습니다."

    # context 생성
    context_parts = []
    for i, (doc, score) in enumerate(filtered, 1):
        context_parts.append(
            f"[{i}] (score={score:.4f}, 출처={doc.metadata.get('출처')})\n"
            f"문의내용: {doc.page_content}\n"
            f"답변내용: {doc.metadata.get('답변내용')}\n"
        )

    context = "\n\n".join(context_parts)

    # 최종 프롬프트
    prompt = (
        "당신은 주어진 컨텍스트만을 바탕으로 한국어로 정확하게 답변하세요.\n"
        "컨텍스트에 없는 내용은 추측하지 말고 '주어진 자료에 근거해 알 수 없습니다'라고 답하세요.\n\n"
        f"사용자 질문:\n{query}\n\n"
        f"--- 컨텍스트 (코사인≥{threshold}) ---\n{context}\n\n"
        "최종 답변:"
    )
    return prompt

def rag_answer(query, k=3, threshold=0.8, model="gpt-4o-mini"):
    # 검색
    results = vectorstore.similarity_search_with_score(query, k=k)

    # 프롬프트 생성
    prompt = build_prompt(query, results, threshold=threshold)

    # OpenAI API 호출
    resp = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
    )
    return resp.choices[0].message.content

# 7. 테스트 실행

## 1. Rag 실행(open ai)

In [88]:
# 테스트 실행
query = "해외발신으로 카드가 발급됐다는 문자를 수신했는데 본인이 신청하지 않았을 경우 사고대응센터에 전화하라고 했어요"
answer = rag_answer(query, k=3, threshold=0.8)
print("질문:", query)
print("\n[생성된 답변]\n", answer)
print("\n[출처]\n", context)

질문: 해외발신으로 카드가 발급됐다는 문자를 수신했는데 본인이 신청하지 않았을 경우 사고대응센터에 전화하라고 했어요

[생성된 답변]
 해외발신으로 카드가 발급됐다는 문자를 수신했지만 본인이 신청하지 않았다면, 해당 카드사의 대표번호로 전화를 하여 발급 유무를 확인해야 합니다. 발신번호나 문자 내 안내처가 아닌 카드사의 공식 번호로 확인하는 것이 중요합니다. 만약 카드사 정보가 없다면, 이는 개인정보 탈취를 시도하는 미끼문자일 가능성이 높습니다. 안드로이드 사용자는 메시지 신고를 통해 피싱으로 신고한 후 문자를 삭제하고 발신번호를 차단하시기 바랍니다. iOS 사용자는 발신번호를 차단한 후 삭제하시기 바랍니다.

[출처]
 [1] (score=0.9489, 출처=문자 2차)
문의내용: 해외발신으로 카드가 발급됐다는 문자를 수신했는데 본인이 신청하지 않았을 경우 사고대응센터에 전화하라고 했어요
답변내용: 정상적인 발급 문자라면 카드사 정보가 있어야 합니다. 발신번호 또는 문자내 안내처가 아닌 해당 카드사의 대표번호로 전화를 하여 발급 유무를 확인해야 하며, 카드사 정보가 없는 경우에는 전화를 유도하여 개인정보 탈취를 하려는 미끼문자일 가능성이 높습니다. 
안드로이드용 사용자는 메시지신고 > 피싱으로 신고 후 문자 삭제를 하고 발신번호도 차단을 하시기 바랍니다. iOS사용자는 발신번호 차단 후 삭제하시기 바랍니다.


[2] (score=0.9190, 출처=문자 2차)
문의내용: 해외발신으로 카드가 발급됐다는 문자를 수신했는데 본인이 신청하지 않았을 경우 전화하라고 해서 통화하여 인증번호를 말했어요
답변내용: 노출된 인증번호의 사용처가 어디인지에 따라 개인정보 노출 피해 상황이 달라질 수 있기 때문에 명의도용 유무 확인방법 1번~3번까지 조치하여 파악하시기 바랍니다.
또한 추후 일어날 수 있는 2차피해 예방을 위해 명의도용 예방법 1번 ~ 4번까지 조치하시기 바랍니다.


[3] (score=0.9059, 출처=문자 2차)
문의내용: 카드가 발급되었다는 해외발신 문

## 2. 그냥 LLM 이용

In [90]:
query

'해외발신으로 카드가 발급됐다는 문자를 수신했는데 본인이 신청하지 않았을 경우 사고대응센터에 전화하라고 했어요'

In [92]:
def answer_only_with_query(query, model="gpt-4o-mini"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": query}],
        temperature=0.7,   # 다양성 정도 (0=사실적, 1=창의적)
    )
    return response.choices[0].message.content

# 테스트 실행
# query = "해외발신으로 카드가 발급됐다는 문자를 수신했는데 본인이 신청하지 않았을 경우 어떻게 해야 하나요?"
answer = answer_only_with_query(query)
print("질문:", query)
print("\n[LLM 답변]\n", answer)

질문: 해외발신으로 카드가 발급됐다는 문자를 수신했는데 본인이 신청하지 않았을 경우 사고대응센터에 전화하라고 했어요

[LLM 답변]
 해외 발신으로 카드가 발급되었다는 문자를 받았고, 본인이 신청하지 않았다면 매우 주의해야 합니다. 이런 경우 사기나 해킹의 위험이 있을 수 있으므로 즉시 다음과 같은 조치를 취하는 것이 좋습니다:

1. **사고대응센터에 연락하기**: 해당 카드사의 사고대응센터에 즉시 전화하여 상황을 설명하고, 카드 발급에 대한 확인을 요청하세요. 카드사에서는 추가적인 조치를 안내해 줄 것입니다.

2. **카드 사용 내역 확인**: 만약 카드가 이미 발급되었다면, 그 카드로의 사용 내역을 확인하여 본인이 모르는 거래가 있는지 체크하세요.

3. **신분 확인**: 카드사에 연락할 때, 본인의 신분을 확인할 수 있는 정보를 미리 준비하세요. 개인 정보가 유출된 경우 추가적인 피해를 방지할 수 있습니다.

4. **신용정보 조회**: 신용정보를 조회하여 본인의 신용카드나 대출에 이상이 없는지 확인하세요.

5. **비밀번호 및 보안 정보 변경**: 만약 카드 정보가 유출되었다고 의심된다면, 관련된 모든 계정의 비밀번호와 보안 정보를 변경하는 것이 좋습니다.

6. **사기 신고**: 필요시 경찰에 사기 신고를 고려하세요.

이런 상황에서는 신속하게 대응하는 것이 중요합니다.


## 3. Ollama + Gemma7b (q4) RAG 함수

In [93]:
import requests

def ollama_generate(model: str, prompt: str, temperature: float = 0.2) -> str:
    """Ollama 로컬 모델 호출 (REST API)"""
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,   # 한 번에 응답 받기
        "options": {"temperature": temperature},
    }
    r = requests.post(url, json=payload, timeout=600)
    r.raise_for_status()
    data = r.json()
    return data.get("response", "").strip()

def rag_answer_ollama(query, k=3, threshold=0.8, 
                      model="gemma:7b-instruct-q4_K_M", temperature=0.2):
    # 1) 벡터 검색
    results = vectorstore.similarity_search_with_score(query, k=k)

    # 2) 프롬프트 만들기 (기존 build_prompt 함수 사용)
    prompt = build_prompt(query, results, threshold=threshold)

    # 3) Ollama 호출
    answer = ollama_generate(model=model, prompt=prompt, temperature=temperature)
    return answer

# 🔹 테스트 실행
query = "해외발신으로 카드가 발급됐다는 문자를 수신했는데 본인이 신청하지 않았을 경우 어떻게 해야 하나요?"
print(rag_answer_ollama(query, k=5, threshold=0.8, model="gemma:7b-instruct-q4_K_M"))

해외발신으로 카드가 발급됐다는 문자를 수신했는데 본인이 신청하지 않았을 경우, 다음과 같은 과정을 거쳐 해결할 수 있습니다.

**1. 카드사 확인:**

* 정상적인 발급 문자라면 카드사 정보가 있어야 합니다.
* 발신번호 또는 문자내 안내처가 아닌 해당 카드사의 대표번호로 전화를 하여 발급 유무를 확인합니다.
* 카드사 정보가 없는 경우에는 전화를 유도하여 개인정보 탈취를 하려는 미끼문자일 가능성이 높습니다.

**2. 피해 예방:**

* 안드로이드용 사용자는 메시지신고 > 피싱으로 신고 후 문자 삭제를 하고 발신번호도 차단을 하시기 바랍니다.
* iOS사용자는 발신번호 차단 후 삭제하시기 바랍니다.
* 명의도용 유무 확인방법 1번~3번까지 조치하여 파악하시기 바랍니다.
* 추후 일어날 수 있는 2차피해 예방을 위해 명의도용 예방법 1번 ~ 4번까지 조치하시기 바랍니다.

**3. 해결:**

* 해외에서 카드 결제 내역이 있다는 문자를 수신하였습니다. 본인이 결제한 것이 아닐 경우 전화하라는 곳이 있어 전화 했는데 이후 금감원과 검찰청에서 전화를 받았습니다.
* 문자에 포함된 전화번호로 연락하게 하여 개인정보를 탈취하거나 악성파일, 원격제어앱 실행을 유도하는 형식의 보이스피싱입니다.
* 현재 검찰청, 금감원 등의 공공기관 대표번호로 전화를 수신하고 소속 공무원을 사칭하는 전화를 받으셨다면 이미 휴대폰은 원격제어 및 악성앱의 영향을 받아 정상적인 전화를 수신, 발신할 수 있는 상태가 아니므로 신속히 휴대폰을 비행기모드 및 데이터차단, 와이파이 차단 모드로 전환하시고 악성설치파일(.apk) 유무를 확인 및 삭제, 모바일 백신 검사를 해야 합니다.


## 4. LangChain + ChatOllama RAG 함수

In [94]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage

# Ollama 로컬 LLM 준비
chat_llm = ChatOllama(
    model="gemma:7b-instruct-q4_K_M",  # ollama list에 있는 이름 그대로
    temperature=0.2,
)

def rag_answer_ollama_langchain(query: str, k: int = 5, threshold: float = 0.8) -> str:
    # 1) 유사 문서 검색 (코사인 점수 포함)
    results = vectorstore.similarity_search_with_score(query, k=k)

    # 2) 프롬프트 생성 (threshold 이상만 컨텍스트로 포함)
    prompt = build_prompt(query, results, threshold=threshold)

    # 3) LLM 호출
    msg = chat_llm.invoke([HumanMessage(content=prompt)])
    return getattr(msg, "content", str(msg))

# 사용 예시
query = "해외발신으로 카드가 발급됐다는 문자를 수신했는데 본인이 신청하지 않았을 경우 어떻게 해야 하나요?"
answer = rag_answer_ollama_langchain(query, k=5, threshold=0.8)
print("[답변]\n", answer)

C:\Users\user\AppData\Local\Temp\ipykernel_22412\1210812642.py:5: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  chat_llm = ChatOllama(


[답변]
 해외발신으로 카드가 발급됐다는 문자를 수신했는데 본인이 신청하지 않았을 경우에는 다음과 같은 과정을 거동합니다.

**1. 카드사 정보 확인:**

* 문자에 포함된 카드사 정보가 정확한지 확인합니다.
* 발신번호 또는 문자내 안내처가 아닌 해당 카드사의 대표번호로 전화를 하여 발급 유무를 확인합니다.
* 카드사 정보가 없는 경우에는 전화를 유도하여 개인정보 탈취를 하려는 미끼문자일 가능성이 높습니다.

**2. 개인정보 노출 피해 예방:**

* 노출된 인증번호의 사용처가 어디인지에 따라 개인정보 노출 피해 상황이 달라질 수 있기 때문에 명의도용 유무 확인방법 1번~3번까지 조치하여 파악하시기 바랍니다.
* 추후 일어날 수 있는 2차피해 예방을 위해 명의도용 예방법 1번 ~ 4번까지 조치하시기 바랍니다.

**3. 해결 방안:**

* 문자에 포함된 전화번호로 연락하게 하여 개인정보를 탈취하거나 악성파일, 원격제어앱 실행을 유도하는 형식의 보이스피싱입니다.
* 현재 검찰청, 금감원 등의 공공기관 대표번호로 전화를 수신하고 소속 공무원을 사칭하는 전화를 받으셨다면 이미 휴대폰은 원격제어 및 악성앱의 영향을 받아 정상적인 전화를 수신, 발신할 수 있는 상태가 아니므로 신속히 휴대폰을 비행기모드 및 데이터차단, 와이파이 차단 모드로 전환하시고악성설치파일(.apk) 유무를 확인 및 삭제, 모바일 백신 검사를 해야합니다.
* 추가적인 금전피해가 발생할 수 있으므로 이용하는 은행의 지급정지 콜센터로 내계좌 일괄 지급정지를 신청하실 수 있습니다.
* 소액결제 피해액에 대해서는 가까운 관할경찰서 민원실에 사건접수를 해주셔야 합니다.
